In [41]:
# Start coding here...
basic_design = data.frame("Month" = rep(c("202207", "202208", "202209", "202210", "202211", "202212", "202301"), 2),
                         "Group" = c(rep("New", 7), rep("Existing", 7)),
                         "Treated" = c(c(0, 0, 0, .5, 1, 1, 1), rep(0, 7)))

In [42]:
basic_design

Month,Group,Treated
<chr>,<chr>,<dbl>
202207,New,0.0
202208,New,0.0
202209,New,0.0
202210,New,0.5
202211,New,1.0
202212,New,1.0
202301,New,1.0
202207,Existing,0.0
202208,Existing,0.0


In [43]:
# Create empty data.frame to fill with disaggregated data 
individual_data = data.frame(
    Month = character(),
    Group = character(),
    Treated = numeric(),
    Dollars = numeric()
)

In [44]:
set.seed(1)
#
for(i in 1:1500){
    # get month-group draw
    month_group_draw = sample(1:nrow(basic_design), 1)
    #
    tmp_mean = 50
    tmp_var = 15
    #
    tmp_mean = ifelse(basic_design[month_group_draw, "Treated"] == 1, 53, 50)
    if(basic_design[month_group_draw, "Treated"] == .5) {
        tmp_mean = 51.5
    }
    if(basic_design[month_group_draw, "Month"] == "202212") {
        tmp_mean = tmp_mean + 3
    }
    if(basic_design[month_group_draw, "Month"] == "202301") {
        tmp_mean = tmp_mean - 4
    }
    tmp_dollars = round(rnorm(1, tmp_mean, tmp_var), 2)
    #
    individual_data[i, ] = cbind(basic_design[month_group_draw, ], "Dollars" = tmp_dollars)
}

In [45]:
# add ids 
treatment_ids = paste(1:200)
control_ids = paste(201:400)
#
individual_data$id = NA
tmp_splitted_individual_data = split(individual_data, f = list(individual_data$Month, individual_data$Group))
#
for(i in 1:length(tmp_splitted_individual_data)) {
    if(tmp_splitted_individual_data[[i]][1, "Group"] == "Existing") {
        tmp_splitted_individual_data[[i]]$id = sample(control_ids, size = nrow(tmp_splitted_individual_data[[i]]), replace = FALSE)
    } else {
        tmp_splitted_individual_data[[i]]$id = sample(treatment_ids, size = nrow(tmp_splitted_individual_data[[i]]), replace = FALSE) 
    }
}
head(tmp_splitted_individual_data[[i]])
#
individual_data = do.call("rbind", tmp_splitted_individual_data)

,Month,Group,Treated,Dollars,id
,<chr>,<chr>,<dbl>,<dbl>,<chr>
7,202301,New,1,55.22,162
20,202301,New,1,69.38,25
26,202301,New,1,60.45,83
36,202301,New,1,58.27,146
40,202301,New,1,59.35,143
46,202301,New,1,39.26,173


In [46]:
tail(individual_data)

,Month,Group,Treated,Dollars,id
,<chr>,<chr>,<dbl>,<dbl>,<chr>
202301.New.1439,202301,New,1,43.23,145
202301.New.1443,202301,New,1,70.89,182
202301.New.1450,202301,New,1,59.21,159
202301.New.1468,202301,New,1,50.63,28
202301.New.1470,202301,New,1,44.44,67
202301.New.1485,202301,New,1,58.35,16


In [47]:
summary(lm(Dollars ~ as.factor(Month) + Group + Treated, data = individual_data))


Call:
lm(formula = Dollars ~ as.factor(Month) + Group + Treated, data = individual_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-47.025 -10.384  -0.549  10.418  49.333 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)             50.5927     1.1957  42.313  < 2e-16 ***
as.factor(Month)202208  -1.2461     1.4940  -0.834  0.40439    
as.factor(Month)202209  -0.9261     1.5009  -0.617  0.53728    
as.factor(Month)202210  -3.0556     1.5516  -1.969  0.04909 *  
as.factor(Month)202211  -1.4591     1.6945  -0.861  0.38933    
as.factor(Month)202212   1.3009     1.6670   0.780  0.43532    
as.factor(Month)202301  -2.3221     1.6636  -1.396  0.16298    
GroupNew                -1.4418     1.1778  -1.224  0.22110    
Treated                  4.4790     1.6980   2.638  0.00843 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15.23 on 1491 degrees of freedom
Multiple R-squared:  0.01729,	Adjus

In [48]:
summary(lm(Dollars ~ as.factor(Month) + Group * Treated, data = individual_data))


Call:
lm(formula = Dollars ~ as.factor(Month) + Group * Treated, data = individual_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-47.025 -10.384  -0.549  10.418  49.333 

Coefficients: (1 not defined because of singularities)
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)             50.5927     1.1957  42.313  < 2e-16 ***
as.factor(Month)202208  -1.2461     1.4940  -0.834  0.40439    
as.factor(Month)202209  -0.9261     1.5009  -0.617  0.53728    
as.factor(Month)202210  -3.0556     1.5516  -1.969  0.04909 *  
as.factor(Month)202211  -1.4591     1.6945  -0.861  0.38933    
as.factor(Month)202212   1.3009     1.6670   0.780  0.43532    
as.factor(Month)202301  -2.3221     1.6636  -1.396  0.16298    
GroupNew                -1.4418     1.1778  -1.224  0.22110    
Treated                  4.4790     1.6980   2.638  0.00843 ** 
GroupNew:Treated             NA         NA      NA       NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’

In [49]:
summary(lm(Dollars ~ as.factor(Month) * Group, data = individual_data))


Call:
lm(formula = Dollars ~ as.factor(Month) * Group, data = individual_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-48.379 -10.429  -0.633  10.448  50.551 

Coefficients:
                                Estimate Std. Error t value Pr(>|t|)    
(Intercept)                       52.349      1.451  36.089  < 2e-16 ***
as.factor(Month)202208            -2.899      2.061  -1.407  0.15972    
as.factor(Month)202209            -3.900      2.056  -1.897  0.05807 .  
as.factor(Month)202210            -6.090      2.042  -2.982  0.00291 ** 
as.factor(Month)202211            -3.293      2.056  -1.601  0.10950    
as.factor(Month)202212            -1.056      2.008  -0.526  0.59922    
as.factor(Month)202301            -2.842      1.996  -1.423  0.15485    
GroupNew                          -5.231      2.131  -2.455  0.01420 *  
as.factor(Month)202208:GroupNew    3.578      2.988   1.197  0.23136    
as.factor(Month)202209:GroupNew    6.361      3.004   2.117  0.03440 *  
as.facto

In [50]:
write.csv(individual_data, "experiment_data.csv", row.names = FALSE)